In [1]:
import pandas as pd
import sklearn as sklearn
import numpy as np
import kaggle
import time
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lg
import optuna
import optuna.integration.lightgbm as lgb
import xgboost as xgb

from sklearn.pipeline import Pipeline
from pandas.core.frame import DataFrame
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from datetime import date
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import RepeatedKFold
from sklearn.svm import LinearSVR
from sklearn.svm import SVR

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## Helper functions

In [3]:
def get_float_cols(df:DataFrame):
    return df.select_dtypes(include=float).columns.tolist()

def get_int_cols(df:DataFrame):
    return df.select_dtypes(include=int).columns.tolist()

def get_number_cols(df:DataFrame):
    return df.select_dtypes(np.number).columns.tolist()

def get_obj_cols(df:DataFrame):
    return list(df.select_dtypes(include=object).columns)

In [4]:
def plot_hist_float(df:DataFrame):
    df_numeric = df[get_float_cols(df)]
    df_numeric.hist(bins=100, figsize=(30,20))
    plt.show()

In [5]:
def plot_hist_int(df:DataFrame):
    df_numeric = df[get_int_cols(df)]
    df_numeric.hist(bins=100, figsize=(30,20))
    plt.show()

In [6]:
def plot_hist_categorical(df:DataFrame):
    
    fig = plt.figure(figsize=(26,150))

    for index, col in enumerate(get_obj_cols(df)):
        plt.subplot(25,2,index+1)
        sns.countplot(x=col, data=df)
        plt.ylabel('COUNT', size = 25)
        plt.xlabel(col, fontsize = 25)
        plt.xticks(size = 20, rotation = 45 )
        plt.yticks(size = 20)
        
    fig.tight_layout(pad=1.0)

In [7]:
def split_test_train(df:DataFrame):
    test, train = df[df['ind'].eq('test')], df[df['ind'].eq('train')]
    test = test.drop(['ind'], axis=1)
    train = train.drop(['ind'], axis=1)
    return test, train
    
def combine_test_train(test:DataFrame, train:DataFrame):
    combine = pd.concat([test.assign(ind='test'), train.assign(ind='train')])
    target = train['SalePrice']
    test_ids = test['Id']
    return combine, target, test_ids

In [8]:
def print_empty_values(df:DataFrame):
    col_names_with_na = list(df.isna().sum()[lambda x: x > 0].index)
    col_names_with_empty = list(df.isnull().sum()[lambda x: x > 0].index)
    result = set(col_names_with_na) | set(col_names_with_empty)    
    print('Columns with NA or empty: {0}'.format(result))

In [9]:
def get_empty_cols(df:DataFrame):
    return list(df.isnull().sum()[lambda x: x > 0].index)

## Data splitting

In [10]:
df_train = pd.read_csv('house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('house-prices-advanced-regression-techniques/test.csv')
df_combine, TARGET, TEST_IDS = combine_test_train(df_test, df_train)

In [11]:
df_combine.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,ind,SalePrice
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,test,NaN
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,test,NaN
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,test,NaN
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal,test,NaN
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal,test,NaN


## Building pipeline

In [12]:
def evaluate_model(predictions, test, test_labels):

    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    mean_error = np.mean(errors)
    
    res = { 'accuracy':accuracy, 'mean': mean_error } 
    return res

In [13]:
class Pipe:
    
    def __init__(self, funcs, **kwargs):
        self.funcs = funcs
        self.kwargs = kwargs
    
    def transform(self, df:DataFrame) -> DataFrame:
        for f in self.funcs:
            df = f(df, **self.kwargs)
            
        return df

In [14]:
class BaseExperiment:
    
    def __init__(self, pipe:Pipe, df:DataFrame, _TARGET_, _TEST_IDS_):
        self.pipe = pipe
        self.df = df
        self._TARGET_ = _TARGET_
        self._TEST_IDS_ = _TEST_IDS_
        self.regressor = self.create_regressor()
        
    def transform(self):        
        self.transformed = self.pipe.transform(self.df)
        return self.transformed 
    
    def create_regressor(self):
        raise NotImplementedError()
    
    def predict_external(self, to_predict):
        return self.regressor.predict(to_predict)
    
    def predict(self):
        test, train = split_test_train(self.transformed)
        
        # fit
        self.regressor.fit(train, self._TARGET_)
        
        # predict
        self.predicted = self.regressor.predict(test)
        
        return self.predicted
    
    def get_metric(self):
        test, train = split_test_train(self.transformed)      
        
        # predict train data
        pred_metric = self.regressor.predict(train)
        
        mse = mean_squared_error(self._TARGET_, pred_metric)
        rmse = np.sqrt(mse)
        
        return rmse

    def to_kaggle(self):
        named_tuple = time.localtime()
        time_string = time.strftime("%m/%d/%Y, %H:%M:%S", named_tuple)
        
        submission = pd.concat([self._TEST_IDS_, pd.Series(self.predicted, name='SalePrice')], axis=1)
        submission.to_csv('./submission.csv', index=False, header=True)
        
        !kaggle competitions submit -c house-prices-advanced-regression-techniques -f submission.csv -m "Test submission"
        
    def get_regressor(self):
        return self.regressor
        

In [15]:
class LinearExperiment(BaseExperiment):
    
    def create_regressor(self):
        return LinearRegression()

In [16]:
class DecisionTreeExperiment(BaseExperiment):
    
    def create_regressor(self):
        return DecisionTreeRegressor()
    
    def get_metric(self):
        test, train = split_test_train(self.transformed)   
        scores = cross_val_score(self.regressor, train, self._TARGET_, scoring='neg_mean_squared_error', cv=10)
        rmse = np.sqrt(-scores)
        return rmse.std(), rmse.mean()

In [17]:
class RandomForestExperiment(DecisionTreeExperiment):
    
    def create_regressor(self):
        return RandomForestRegressor()

In [18]:
class RandomForestWithGridSearchExperiment(RandomForestExperiment):
    
    best_params = { 
        'n_estimators': 400,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'max_features': 'sqrt',
        'max_depth': 90, 
        'bootstrap': False }
    
    def create_regressor(self):
        return RandomForestRegressor(**self.best_params)    
    
    def search_best_params(self):
        test, train = split_test_train(self.transformed)
        
        long_param_grid = {
            'bootstrap': [True, False],
            'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
            'max_features': ['auto', 'sqrt'],
            'min_samples_leaf': [1, 2, 4],
            'min_samples_split': [2, 5, 10],
            'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000] }
           
        rf_random = RandomizedSearchCV(estimator = self.regressor,
                                       param_distributions = long_param_grid,
                                       n_iter = 100, 
                                       cv = 3, 
                                       verbose=2,
                                       random_state=42,
                                       n_jobs = -1)
        
        # fit
        rf_random.fit(train, self._TARGET_)

        # get best params
        self.best_params = rf_random.best_params_
                
        return self.best_params


In [19]:
class LightGBMExperiment(BaseExperiment):
    
    def create_regressor(self):
        return lg.LGBMRegressor()

In [20]:
class LightGBMExperimentWithParams(BaseExperiment):

    best_params = {
        'objective': 'regression', 
        'metric': 'rmse', 
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'seed': 42, 
        'feature_pre_filter': False, 
        'lambda_l1': 7.089889561174952e-07,
        'lambda_l2': 0.02142265423635661,
        'num_leaves': 10, 
        'feature_fraction': 0.4, 
        'bagging_fraction': 0.9990730129106954,
        'bagging_freq': 4, 
        'min_child_samples': 5}
    
    def create_regressor(self):
        return lg.LGBMRegressor(**self.best_params)
    
    def search_best_params(self):
        
        rkf = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

        params = {
            "objective": "regression",
            "metric": "rmse",
            "verbosity": -1,
            "boosting_type": "gbdt",                
            "seed": 42 }
        
        test, train = split_test_train(self.transformed)
        X = train
        y = self._TARGET_
        dtrain = lgb.Dataset(X, label=y)
        
        study_tuner = optuna.create_study(direction='minimize')        
        optuna.logging.set_verbosity(optuna.logging.WARNING) 
        
        tuner = lgb.LightGBMTunerCV(params, 
                            dtrain, 
                            study=study_tuner,
                            seed = 42,
                            folds=rkf,
                            num_boost_round=1000                       
                            )

        tuner.run()        
        self.best_params = tuner.best_params        
        return tuner

In [21]:
class XGBExperiment(BaseExperiment):
    
    def create_regressor(self):
        return xgb.XGBRegressor()

In [22]:
class XGBExperimentWithParams(BaseExperiment):
    
    best_params = {
        'max_depth': 7, 
        'learning_rate': 0.07565159949506778,
        'n_estimators': 714, 
        'min_child_weight': 6,
        'gamma': 0.9324748648192458, 
        'subsample': 0.14074600746618596, 
        'colsample_bytree': 0.6983721197854935, 
        'reg_alpha': 0.10885510377662143,
        'reg_lambda': 0.7756168972298106, 
        'random_state': 356 }
    
    def create_regressor(self):
        return xgb.XGBRegressor(**self.best_params)
    
    def transform(self):        
        self.transformed = self.pipe.transform(self.df)
        
        # static for objective function
        XGBExperimentWithParams.TRANSFORM = self.transformed
        XGBExperimentWithParams.TARGET = self._TARGET_
        
        return self.transformed 
    
    def objective(trial):
        
        param = {
            'max_depth': trial.suggest_int('max_depth', 1, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
            'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'gamma': trial.suggest_float('gamma', 0.01, 1.0),
            'subsample': trial.suggest_float('subsample', 0.01, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
            'random_state': trial.suggest_int('random_state', 1, 1000) }
        
        # split
        test, train = split_test_train(XGBExperimentWithParams.TRANSFORM)
        
        # fit
        model = xgb.XGBRegressor(**param)
        model.fit(train, XGBExperimentWithParams.TARGET)
        
        # get score
        scores = cross_val_score(model, train, XGBExperimentWithParams.TARGET, scoring='neg_mean_squared_error', cv=3)
        rmse = np.sqrt(-scores)
        
        return np.mean(rmse)
    
    def search_best_params(self):
        
        study = optuna.create_study(direction='minimize', study_name='regression')
        study.optimize(XGBExperimentWithParams.objective, n_trials=100)
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        self.best_param = study.best_param
        return study.best_params

In [23]:
class LinearSVRExperiment(BaseExperiment):
    
    def create_regressor(self):
        return LinearSVR(epsilon=1.5)

In [24]:
class SVRExperiment(BaseExperiment):
    
    def create_regressor(self):
        return SVR(kernel='poly', degree=2, C=100, epsilon=0.1)

In [25]:
class CatBoostExperiment(BaseExperiment):
    
    def create_regressor(self):
        return CatBoostRegressor()

In [26]:
class CatBoostWithParamsExperiment(BaseExperiment):
    
    best_params =  {'iterations': 250, 
                    'learning_rate': 0.1736687160463188,
                    'depth': 11}
    
    
    def create_regressor(self):
        return CatBoostRegressor(**self.best_params)

    def transform(self):        
        self.transformed = self.pipe.transform(self.df)

        # static for objective function
        CatBoostWithParamsExperiment.TRANSFORM = self.transformed
        CatBoostWithParamsExperiment.TARGET = self._TARGET_

        return self.transformed 
    
    def objective(trial):
        
        param = {
            'iterations': trial.suggest_int('iterations', 1, 1000),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
            'depth': trial.suggest_int('depth', 2, 16) }

        # split
        test, train = split_test_train(CatBoostWithParamsExperiment.TRANSFORM)
        
        # fit
        model = CatBoostRegressor(**param)
        model.fit(train, CatBoostWithParamsExperiment.TARGET)
        
        # get score
        scores = cross_val_score(model, train, CatBoostWithParamsExperiment.TARGET, scoring='neg_mean_squared_error', cv=3)
        rmse = np.sqrt(-scores)        
        return np.mean(rmse)
    
    def search_best_params(self):
        
        study = optuna.create_study(direction='minimize', study_name='regression')
        study.optimize(CatBoostWithParamsExperiment.objective, n_trials=50)
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        self.best_param = study.best_param
        return study.best_params
    
    def get_metric_cross_val(self):
        test, train = split_test_train(self.transformed)   
        scores = cross_val_score(self.regressor, train, self._TARGET_, scoring='neg_mean_squared_error', cv=10)
        rmse = np.sqrt(-scores)
        return rmse.std(), rmse.mean()

In [27]:
def impute_categorical_with_na_avail(df:DataFrame, **kwargs) -> DataFrame:
        
    cols_to_impute = [
        'Alley',
        'BsmtQual',
        'BsmtCond',
        'BsmtExposure',
        'BsmtFinType1',
        'BsmtFinType2',
        'FireplaceQu',
        'GarageType',
        'GarageFinish',
        'GarageQual',
        'GarageCond',
        'PoolQC',
        'Fence',
        'MiscFeature']
    
    for col in cols_to_impute:
        df[col] = df[col].fillna('NA')
        
    return df

In [28]:
def impute_categorical_with_na_not_avail(df:DataFrame, **kwargs) -> DataFrame:
        
    cols_to_impute = [ 
        'MSZoning',
        'Utilities',
        'Exterior1st',
        'Exterior2nd',
        'MasVnrType',
        'Electrical',
        'KitchenQual',
        'Functional',
        'SaleType']
    
    for col in cols_to_impute:
        df[col] = df[col].mode()[0]
        
    return df

In [29]:
def delete_not_needed(df:DataFrame, **kwargs) -> DataFrame:
    to_delete = [ 'Id', 'SalePrice' ]
    for d in to_delete:
        if d in df.columns:
            df = df.drop(columns=[d])
            
    return df

In [30]:
def add_features(df:DataFrame, **kwargs) -> DataFrame:
    
    # square per room 
    df["SqFtPerRoom"] = df["GrLivArea"] / (df["TotRmsAbvGrd"] + df["FullBath"] + df["HalfBath"] + df["KitchenAbvGr"])
    
    # quality
    df['Total_Home_Quality'] = df['OverallQual'] + df['OverallCond']
    
    # bathrooms
    df['Total_Bathrooms'] = (df['FullBath'] + (0.5 * df['HalfBath']) + df['BsmtFullBath'] + (0.5 * df['BsmtHalfBath']))
    
    # sum of squre of 1 and 2 floor
    df["HighQualSF"] = df["1stFlrSF"] + df["2ndFlrSF"]
    return df

In [31]:
def remove_skew(df:DataFrame, **kwargs) -> DataFrame:
    
    skew_df = pd.DataFrame(get_number_cols(df), columns=['Feature'])
    skew_df['Skew'] = skew_df['Feature'].apply(lambda feature: scipy.stats.skew(df[feature]))
    skew_df['Absolute Skew'] = skew_df['Skew'].apply(abs)
    skew_df['Skewed'] = skew_df['Absolute Skew'].apply(lambda x: True if x >= 0.5 else False)

    for column in skew_df.query("Skewed == True")['Feature'].values:
        df[column] = np.log1p(df[column])
    
    return df

In [32]:
def fix_subclass(df:DataFrame, **kwargs) -> DataFrame:
    df['MSSubClass'] = df['MSSubClass'].astype(str)
    return df

In [33]:
def fix_month_sold(df:DataFrame, **kwargs) -> DataFrame:
    df['MoSold'] = (-np.cos(df['MoSold']))
    return df

In [34]:
def scale(df:DataFrame, **kwargs) -> DataFrame:
    
    scaling_type = kwargs['scaling_type']
    
    if (scaling_type == 'MinMaxScaler'):
        scaler = MinMaxScaler()
    
    if (scaling_type == 'MaxAbsScaler'):
        scaler = MaxAbsScaler()
    
    if (scaling_type == 'StandardScaler'):
        scaler = StandardScaler()
    
    if (scaling_type == 'RobustScaler'):
        scaler = RobustScaler()        
    
    # save ind column
    test_train_mask = df['ind']
    df = df.drop(['ind'], axis=1)
    
    scaler.fit(df)
    df = pd.DataFrame(scaler.transform(df), index=df.index, columns=df.columns) 
    
    # restore column
    df['ind'] = test_train_mask
    
    return df

In [35]:
def one_hot_encoding(df:DataFrame, **kwargs) -> DataFrame:
    
    test_train_mask = df['ind']
    df = df.drop(['ind'], axis=1)
    df = pd.get_dummies(df)
    df['ind'] = test_train_mask
    return df

In [36]:
def impute_numeric_cols(df:DataFrame, **kwargs) -> DataFrame:
    
    impute_method = kwargs['impute_method']
    
    col_names = [
        'LotFrontage',
        'MasVnrArea',
        'BsmtFinSF1',
        'BsmtFinSF2',
        'BsmtUnfSF',
        'TotalBsmtSF',
        'BsmtFullBath',
        'BsmtHalfBath',
        'GarageYrBlt',
        'GarageCars',
        'GarageArea' ]
    
    # all possible columns
    initial_cols = list(df.columns)
    
    for col_name in col_names:

        non_empty_numeric =  (set(get_number_cols(df)) - set(get_empty_cols(df))) | {col_name}        
        cols_names_to_drop = list((set(df.columns) - non_empty_numeric))
        
        if (kwargs['verbose']==True): print('Deleting: {0}. Imputing: {1}'.format(cols_names_to_drop, col_name))
        
        # save temp
        temp = df[cols_names_to_drop]

        # clear dataset 
        df = df.drop(columns=cols_names_to_drop)
        
        known = df.loc[ df[col_name].notnull() ]        
        unknown = df.loc[ df[col_name].isnull() ]
        
        # nothing to predict
        if (len(unknown) == 0): 
            if (kwargs['verbose']==True): print('Nothing to predict - continue')
            continue
        
        column_index = list(df.columns).index(col_name)

        all_indices = [i for i in range(unknown.shape[1])]
        diff = list(set(all_indices) - {column_index})

        y = known.values[:, column_index]
        X = known.values[:, diff]

        # select regressor
        if impute_method == 'randomforest':
            regressor = RandomForestRegressor(n_estimators=100, n_jobs=-1)
        if impute_method == 'knn':
            regressor = KNeighborsRegressor()
            
        regressor.fit(X, y)
        predicted = regressor.predict(unknown.values[:, diff])
        
        if (kwargs['verbose']==True): print('{0} was predicted. Len: {1}'.format(col_name, len(predicted)))
        
        # fill missings
        df.loc[ (df[col_name].isnull()), col_name ] = predicted
        
        # restore dataset
        df[cols_names_to_drop] = temp
    
    # reorder columns back
    df = df.reindex(columns = initial_cols)
    
    return df

## Plotting

In [37]:
#plot_hist_float(df_combine)

In [38]:
#plot_hist_int(df_combine)

In [39]:
#plot_hist_categorical(df_combine)

## Defining pipes

In [40]:
random_forest_imputer_pipe = Pipe([
        fix_subclass,
        impute_categorical_with_na_avail,
        impute_categorical_with_na_not_avail,
        delete_not_needed,
        impute_numeric_cols,
        one_hot_encoding
        ],
        
        verbose = False,
        impute_method = 'randomforest'
    )

knn_pipe = Pipe([
        fix_subclass,
        impute_categorical_with_na_avail,
        impute_categorical_with_na_not_avail,
        delete_not_needed,
        impute_numeric_cols,
        one_hot_encoding
        ],
        
        verbose = False,
        impute_method = 'knn'
    )

no_skew_pipe = Pipe([
        fix_subclass,
        impute_categorical_with_na_avail,
        impute_categorical_with_na_not_avail,
        delete_not_needed,
        impute_numeric_cols,
        one_hot_encoding,
        remove_skew
        ],
        
        verbose = False,
        impute_method = 'knn'
    )

no_skew_scaling = Pipe([
        fix_subclass,
        impute_categorical_with_na_avail,
        impute_categorical_with_na_not_avail,
        delete_not_needed,
        impute_numeric_cols,
        one_hot_encoding,
        remove_skew,
        scale
        ],
        
        verbose = False,
        impute_method = 'knn',
        scaling_type='RobustScaler'
    )

no_skew_scaling_cosine = Pipe([
        fix_subclass,
        fix_month_sold,
        impute_categorical_with_na_avail,
        impute_categorical_with_na_not_avail,
        delete_not_needed,
        impute_numeric_cols,
        one_hot_encoding,
        remove_skew,
        scale
        ],
        
        verbose = False,
        impute_method = 'knn',
        scaling_type='RobustScaler'
    )

no_skew_scaling_cosine_more_features = Pipe([
        fix_month_sold,
        impute_categorical_with_na_avail,
        impute_categorical_with_na_not_avail,
        delete_not_needed,
        impute_numeric_cols,
        add_features,
        one_hot_encoding,
        remove_skew,
        scale
        ],
        
        verbose = False,
        impute_method = 'randomforest',
        scaling_type='RobustScaler'
    )


## Liniear regression

In [41]:
experiment = LinearExperiment(random_forest_imputer_pipe, df_combine, TARGET, TEST_IDS)
experiment.transform()
predicted = experiment.predict()
metric = experiment.get_metric()
print('RMSE: {0}'.format(metric))

RMSE: 21607.0002378834


In [42]:
#experiment = LinearExperiment(knn_pipe, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))

In [43]:
#experiment = LinearExperiment(no_skew_pipe, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))

In [44]:
#experiment = LinearExperiment(no_skew_scaling, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))

## Decision Tree

In [45]:
#experiment = DecisionTreeExperiment(no_skew_pipe, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0:.4f} STD: {1:.2f}'.format(metric[1], metric[0]))

In [46]:
#experiment = DecisionTreeExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0:.4f} STD: {1:.2f}'.format(metric[1], metric[0]))

## Random Forest

In [47]:
#experiment = RandomForestExperiment(no_skew_scaling, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0:.4f} STD: {1:.2f}'.format(metric[1], metric[0]))

In [48]:
#experiment = RandomForestExperiment(no_skew_scaling_cosine, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0:.4f} STD: {1:.2f}'.format(metric[1], metric[0]))

In [49]:
#experiment = RandomForestExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0:.4f} STD: {1:.2f}'.format(metric[1], metric[0]))

## Random Forest with best params

In [50]:
#experiment = RandomForestWithGridSearchExperiment(no_skew_scaling, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0:.4f} STD: {1:.2f}'.format(metric[1], metric[0]))

In [51]:
#experiment = RandomForestWithGridSearchExperiment(no_skew_scaling_cosine, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0:.4f} STD: {1:.2f}'.format(metric[1], metric[0]))

In [52]:
#experiment = RandomForestWithGridSearchExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0:.4f} STD: {1:.2f}'.format(metric[1], metric[0]))

## Search params for RandomForest

In [53]:
#experiment = RandomForestWithGridSearchExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#p = experiment.search_best_params()
#print(p)

## LightGBM

In [54]:
#experiment = LightGBMExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))
#experiment.to_kaggle()

In [55]:
#experiment = LightGBMExperimentWithParams(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))

## XGB with no params

In [56]:
#experiment = XGBExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))
#experiment.to_kaggle()

In [57]:
#experiment = XGBExperimentWithParams(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))

## SVR

In [58]:
#experiment = LinearSVRExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))

In [59]:
#experiment = SVRExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))

## CatBoost

In [60]:
#experiment = CatBoostExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))

In [ ]:
experiment = CatBoostWithParamsExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
experiment.transform()
p = experiment.search_best_params()
print('Params: {0}'.format(p))

[I 2023-05-04 11:20:00,711] A new study created in memory with name: regression


0:	learn: 43539.9213866	total: 165ms	remaining: 1m 32s
1:	learn: 34526.3908057	total: 183ms	remaining: 51.3s
2:	learn: 30580.2051632	total: 200ms	remaining: 37.3s
3:	learn: 28450.9965623	total: 220ms	remaining: 30.8s
4:	learn: 27719.0903021	total: 238ms	remaining: 26.6s
5:	learn: 25942.9878365	total: 258ms	remaining: 24s
6:	learn: 24662.4825012	total: 277ms	remaining: 22s
7:	learn: 23517.0505330	total: 295ms	remaining: 20.5s
8:	learn: 23262.6736748	total: 310ms	remaining: 19.1s
9:	learn: 22360.3242571	total: 326ms	remaining: 18.1s
10:	learn: 21265.8814858	total: 341ms	remaining: 17.1s
11:	learn: 19909.8424711	total: 359ms	remaining: 16.5s
12:	learn: 18562.9751628	total: 382ms	remaining: 16.2s
13:	learn: 17646.2775947	total: 411ms	remaining: 16.1s
14:	learn: 17113.9547371	total: 431ms	remaining: 15.8s
15:	learn: 16614.2240547	total: 449ms	remaining: 15.4s
16:	learn: 15700.9484688	total: 464ms	remaining: 14.9s
17:	learn: 15170.1549412	total: 479ms	remaining: 14.5s
18:	learn: 14847.441811

160:	learn: 782.7795744	total: 3.22s	remaining: 8.07s
161:	learn: 763.2035732	total: 3.24s	remaining: 8.05s
162:	learn: 752.9306033	total: 3.26s	remaining: 8.03s
163:	learn: 742.9438404	total: 3.28s	remaining: 8s
164:	learn: 729.9768365	total: 3.29s	remaining: 7.97s
165:	learn: 715.0259788	total: 3.31s	remaining: 7.93s
166:	learn: 713.1368980	total: 3.32s	remaining: 7.9s
167:	learn: 699.6100393	total: 3.34s	remaining: 7.87s
168:	learn: 689.1852316	total: 3.35s	remaining: 7.84s
169:	learn: 677.2762455	total: 3.37s	remaining: 7.82s
170:	learn: 670.6584248	total: 3.39s	remaining: 7.79s
171:	learn: 660.0014633	total: 3.41s	remaining: 7.76s
172:	learn: 645.7336496	total: 3.42s	remaining: 7.74s
173:	learn: 639.0846913	total: 3.44s	remaining: 7.71s
174:	learn: 620.8626880	total: 3.46s	remaining: 7.68s
175:	learn: 603.0628760	total: 3.47s	remaining: 7.66s
176:	learn: 587.4894875	total: 3.49s	remaining: 7.63s
177:	learn: 576.3733517	total: 3.5s	remaining: 7.6s
178:	learn: 565.4095998	total: 3.5

317:	learn: 60.9425645	total: 5.83s	remaining: 4.51s
318:	learn: 60.5740853	total: 5.84s	remaining: 4.49s
319:	learn: 59.7199191	total: 5.86s	remaining: 4.46s
320:	learn: 58.7235873	total: 5.87s	remaining: 4.44s
321:	learn: 57.7790489	total: 5.88s	remaining: 4.42s
322:	learn: 56.8064215	total: 5.9s	remaining: 4.4s
323:	learn: 54.8771103	total: 5.92s	remaining: 4.38s
324:	learn: 54.3095526	total: 5.93s	remaining: 4.36s
325:	learn: 53.8572371	total: 5.94s	remaining: 4.34s
326:	learn: 53.0801552	total: 5.96s	remaining: 4.32s
327:	learn: 52.7317838	total: 5.97s	remaining: 4.3s
328:	learn: 51.9053789	total: 5.99s	remaining: 4.28s
329:	learn: 51.1442148	total: 6.01s	remaining: 4.26s
330:	learn: 50.2756694	total: 6.02s	remaining: 4.24s
331:	learn: 48.9034696	total: 6.04s	remaining: 4.22s
332:	learn: 48.5160438	total: 6.05s	remaining: 4.2s
333:	learn: 47.9082905	total: 6.07s	remaining: 4.18s
334:	learn: 47.2520598	total: 6.09s	remaining: 4.16s
335:	learn: 46.7690362	total: 6.1s	remaining: 4.14

478:	learn: 5.5531755	total: 8.37s	remaining: 1.48s
479:	learn: 5.4990875	total: 8.38s	remaining: 1.47s
480:	learn: 5.4355902	total: 8.39s	remaining: 1.45s
481:	learn: 5.3532417	total: 8.41s	remaining: 1.43s
482:	learn: 5.2327693	total: 8.42s	remaining: 1.41s
483:	learn: 5.1430633	total: 8.43s	remaining: 1.39s
484:	learn: 5.1368794	total: 8.44s	remaining: 1.38s
485:	learn: 5.0096583	total: 8.46s	remaining: 1.36s
486:	learn: 4.8981390	total: 8.47s	remaining: 1.34s
487:	learn: 4.8593773	total: 8.48s	remaining: 1.32s
488:	learn: 4.8051154	total: 8.49s	remaining: 1.3s
489:	learn: 4.8011430	total: 8.51s	remaining: 1.28s
490:	learn: 4.7506074	total: 8.52s	remaining: 1.27s
491:	learn: 4.6303717	total: 8.54s	remaining: 1.25s
492:	learn: 4.5460134	total: 8.55s	remaining: 1.23s
493:	learn: 4.4619963	total: 8.56s	remaining: 1.21s
494:	learn: 4.3719610	total: 8.58s	remaining: 1.2s
495:	learn: 4.3141329	total: 8.59s	remaining: 1.18s
496:	learn: 4.2668787	total: 8.6s	remaining: 1.16s
497:	learn: 4.2

...

289:	learn: 0.0496296	total: 3m 11s	remaining: 4m 20s
290:	learn: 0.0484818	total: 3m 11s	remaining: 4m 20s
291:	learn: 0.0460813	total: 3m 12s	remaining: 4m 19s
292:	learn: 0.0430108	total: 3m 13s	remaining: 4m 19s
293:	learn: 0.0400541	total: 3m 13s	remaining: 4m 18s
294:	learn: 0.0392344	total: 3m 14s	remaining: 4m 17s
295:	learn: 0.0369847	total: 3m 15s	remaining: 4m 17s
296:	learn: 0.0357054	total: 3m 15s	remaining: 4m 16s
297:	learn: 0.0350816	total: 3m 16s	remaining: 4m 15s
298:	learn: 0.0325588	total: 3m 17s	remaining: 4m 15s
299:	learn: 0.0312990	total: 3m 17s	remaining: 4m 14s
300:	learn: 0.0290716	total: 3m 18s	remaining: 4m 13s
301:	learn: 0.0278675	total: 3m 19s	remaining: 4m 13s
302:	learn: 0.0261526	total: 3m 19s	remaining: 4m 12s
303:	learn: 0.0252842	total: 3m 20s	remaining: 4m 11s
304:	learn: 0.0251578	total: 3m 21s	remaining: 4m 11s
305:	learn: 0.0243290	total: 3m 21s	remaining: 4m 10s
306:	learn: 0.0228405	total: 3m 22s	remaining: 4m 10s
307:	learn: 0.0222793	total:

442:	learn: 0.0001821	total: 4m 52s	remaining: 2m 40s
443:	learn: 0.0001810	total: 4m 52s	remaining: 2m 39s
444:	learn: 0.0001749	total: 4m 53s	remaining: 2m 38s
445:	learn: 0.0001695	total: 4m 54s	remaining: 2m 38s
446:	learn: 0.0001631	total: 4m 54s	remaining: 2m 37s
447:	learn: 0.0001571	total: 4m 55s	remaining: 2m 36s
448:	learn: 0.0001517	total: 4m 56s	remaining: 2m 36s
449:	learn: 0.0001501	total: 4m 56s	remaining: 2m 35s
450:	learn: 0.0001457	total: 4m 57s	remaining: 2m 34s
451:	learn: 0.0001428	total: 4m 57s	remaining: 2m 34s
452:	learn: 0.0001368	total: 4m 58s	remaining: 2m 33s
453:	learn: 0.0001364	total: 4m 59s	remaining: 2m 32s
454:	learn: 0.0001331	total: 4m 59s	remaining: 2m 32s
455:	learn: 0.0001308	total: 5m	remaining: 2m 31s
456:	learn: 0.0001219	total: 5m 1s	remaining: 2m 31s
457:	learn: 0.0001140	total: 5m 2s	remaining: 2m 30s
458:	learn: 0.0001086	total: 5m 2s	remaining: 2m 29s
459:	learn: 0.0001012	total: 5m 3s	remaining: 2m 29s
460:	learn: 0.0000974	total: 5m 4s	r

596:	learn: 0.0000007	total: 6m 34s	remaining: 58.9s
597:	learn: 0.0000006	total: 6m 36s	remaining: 58.4s
598:	learn: 0.0000006	total: 6m 38s	remaining: 57.9s
599:	learn: 0.0000006	total: 6m 39s	remaining: 57.2s
600:	learn: 0.0000006	total: 6m 40s	remaining: 56.6s
601:	learn: 0.0000006	total: 6m 42s	remaining: 56.2s
602:	learn: 0.0000005	total: 6m 43s	remaining: 55.6s
603:	learn: 0.0000005	total: 6m 44s	remaining: 54.9s
604:	learn: 0.0000005	total: 6m 45s	remaining: 54.3s
605:	learn: 0.0000005	total: 6m 46s	remaining: 53.6s
606:	learn: 0.0000005	total: 6m 46s	remaining: 53s
607:	learn: 0.0000004	total: 6m 47s	remaining: 52.3s
608:	learn: 0.0000004	total: 6m 48s	remaining: 51.6s
609:	learn: 0.0000004	total: 6m 49s	remaining: 51s
610:	learn: 0.0000004	total: 6m 49s	remaining: 50.3s
611:	learn: 0.0000004	total: 6m 50s	remaining: 49.6s
612:	learn: 0.0000004	total: 6m 51s	remaining: 49s
613:	learn: 0.0000004	total: 6m 52s	remaining: 48.3s
614:	learn: 0.0000004	total: 6m 52s	remaining: 47.7s

62:	learn: 863.4242963	total: 1m 25s	remaining: 14m 9s
63:	learn: 848.4295524	total: 1m 27s	remaining: 14m 6s
64:	learn: 769.5746005	total: 1m 28s	remaining: 14m 5s
65:	learn: 719.1715795	total: 1m 29s	remaining: 14m 4s
66:	learn: 686.0805008	total: 1m 31s	remaining: 14m 5s
67:	learn: 633.9219256	total: 1m 33s	remaining: 14m 5s
68:	learn: 603.7331216	total: 1m 34s	remaining: 14m 4s
69:	learn: 568.5759331	total: 1m 35s	remaining: 14m 3s
70:	learn: 544.9654567	total: 1m 37s	remaining: 14m 2s
71:	learn: 513.5243673	total: 1m 38s	remaining: 14m 1s
72:	learn: 481.7873496	total: 1m 40s	remaining: 13m 59s
73:	learn: 459.5715373	total: 1m 41s	remaining: 13m 58s
74:	learn: 437.3711478	total: 1m 42s	remaining: 13m 56s
75:	learn: 404.8750694	total: 1m 44s	remaining: 13m 55s
76:	learn: 378.4262759	total: 1m 45s	remaining: 13m 54s
77:	learn: 364.2422057	total: 1m 47s	remaining: 13m 55s
78:	learn: 350.5819923	total: 1m 48s	remaining: 13m 55s
79:	learn: 334.3478128	total: 1m 50s	remaining: 13m 54s
80

211:	learn: 0.7777082	total: 4m 19s	remaining: 9m 39s
212:	learn: 0.7390005	total: 4m 20s	remaining: 9m 37s
213:	learn: 0.7010363	total: 4m 22s	remaining: 9m 37s
214:	learn: 0.6851166	total: 4m 23s	remaining: 9m 36s
215:	learn: 0.6566496	total: 4m 24s	remaining: 9m 34s
216:	learn: 0.6130096	total: 4m 25s	remaining: 9m 33s
217:	learn: 0.5859158	total: 4m 25s	remaining: 9m 30s
218:	learn: 0.5816828	total: 4m 26s	remaining: 9m 28s
219:	learn: 0.5583380	total: 4m 27s	remaining: 9m 26s
220:	learn: 0.5479213	total: 4m 28s	remaining: 9m 24s
221:	learn: 0.5371114	total: 4m 29s	remaining: 9m 22s
222:	learn: 0.5312340	total: 4m 30s	remaining: 9m 21s
223:	learn: 0.4989190	total: 4m 31s	remaining: 9m 19s
224:	learn: 0.4767226	total: 4m 31s	remaining: 9m 17s
225:	learn: 0.4712510	total: 4m 32s	remaining: 9m 13s
226:	learn: 0.4401909	total: 4m 32s	remaining: 9m 11s
227:	learn: 0.4351118	total: 4m 33s	remaining: 9m 9s
228:	learn: 0.4178900	total: 4m 34s	remaining: 9m 7s
229:	learn: 0.3856708	total: 4

In [ ]:
#experiment = CatBoostWithParamsExperiment(no_skew_scaling_cosine_more_features, df_combine, TARGET, TEST_IDS)
#experiment.transform()
#predicted = experiment.predict()
#metric = experiment.get_metric()
#print('RMSE: {0}'.format(metric))
#experiment.to_kaggle()